# 张艺馨 15210130100

## 1.抓取47年政府工作报告

In [1]:
import urllib2
from bs4 import BeautifulSoup

In [2]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://www.hprc.org.cn/wxzl/wxysl/lczf/ width=1000 height=500></iframe>')

In [8]:
url ="http://www.hprc.org.cn/wxzl/wxysl/lczf/"
content=urllib2.urlopen(url).read().decode('gb18030')
soup= BeautifulSoup(content,'html.parser')
links=soup.find_all('td',{'class','bl'})
print len(links)

47


In [12]:
hyperlinks=[url+i.a['href'].split('./')[1] for i in links]

In [15]:
hyperlinks

[u'http://www.hprc.org.cn/wxzl/wxysl/lczf/d12qgrdzfbg/201603/t20160318_369509.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/d12qgrdzfbg/201503/t20150318_319434.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/d12qgrdzfbg/201403/t20140315_270863.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/d12qgrdzfbg/201402/t20140214_266528.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie/201402/t20140214_266527.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie/201103/t20110315_153641.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie/201003/t20100315_44772.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie/200908/t20090817_27504.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie/200908/t20090817_27495.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie_1/200908/t20090818_27775.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie_1/200908/t20090818_27765.html',
 u'http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie_1/200908/t20090818_2

In [18]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie/201003/t20100315_44772.html \
width=1000 height=500></iframe>')

In [26]:
url_i="http://www.hprc.org.cn/wxzl/wxysl/lczf/dishiyijie/201003/t20100315_44772.html"
content = urllib2.urlopen(url_i).read().decode('gb18030')
soup = BeautifulSoup(content,'html.parser')
scripts=soup.find_all('script')

In [28]:
countPage= int(''.join(scripts[1]).split('countPage = ')[1].split('//')[0])
countPage

6

In [ ]:
def crawler(url_i):
    content = urllib2.urlopen(url_i).read().decode('gb18030')  
    soup = BeautifulSoup(content, 'html.parser') 
    year = soup.find('span', {'class', 'huang16c'}).text[:4]
    year = int(year)
    report = ''.join(s.text for s in soup('p'))
    # 找到分页信息
    scripts = soup.find_all('script')
    countPage = int(''.join(scripts[1]).split('countPage = ')[1].split('//')[0])
    if countPage == 1:
        pass
    else:
        for i in range(1, countPage):
            url_child = url_i.split('.html')[0] +'_'+str(i)+'.html'
            content = urllib2.urlopen(url_child).read().decode('gb18030') 
            soup = BeautifulSoup(content) 
            report_child = ''.join(s.text for s in soup('p'))
            report = report + report_child
    return year, report

## ——生成47年政府工作报告

In [37]:
reports = {}
for link in hyperlinks:
    year, report = crawler(link)
    reports[year] = report 
with open ('/Users/zhangyixin/Desktop/cjc2016-gh-pages/data/gov_reports1954-2016.txt','wb') as f:
    for r in reports:
        line =str(r)+'\t'+reports[r].replace('\n','\t')+'\n'
        f.write(line.encode('utf-8'))

## 2.抓取天涯论坛数据(以NBA为关键词)

## 2.1

In [1]:
import urllib2
from bs4 import BeautifulSoup

In [2]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=NBA width=1000 height=500></iframe>')

In [39]:
page_num = 0
url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=NBA" % page_num
content = urllib2.urlopen(url).read() 
soup = BeautifulSoup(content, "lxml") 
articles = soup.find_all('tr')

In [40]:
len(articles)

51

In [41]:
def crawler(page_num, file_name):
    try:
        url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=NBA" % page_num
        content = urllib2.urlopen(url).read() 
        soup = BeautifulSoup(content, "lxml") 
        articles = soup.find_all('tr')
        for i in articles[1:]:
            td = i.find_all('td')
            title = td[0].text.strip()
            title_url = td[0].a['href']
            author = td[1].text
            author_url = td[1].a['href']
            views = td[2].text
            replies = td[3].text
            date = td[4]['title']
            record = title + '\t' + title_url+ '\t' + author + '\t'+ \
                        author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
            with open(file_name,'a') as p: 
                        p.write(record.encode('utf-8')+"\n") 
    except Exception, e:
        print e
        pass

In [42]:
for page_num in range(10):
    print (page_num)
    crawler(page_num, '/Users/zhangyixin/Documents/tianya_bbs_threads_list.txt') 

0
1
2
3
4
5
6
7
8
9


In [7]:
import pandas as pd
df = pd.read_csv('/Users/zhangyixin/Documents/tianya_bbs_threads_list.txt', sep = "\t", header=None)
df[:5]

,0,1,2,3,4,5,6
0,"从抵制家乐福到排斥NBA,中国又不高兴了?",/post-free-1858023-1.shtml,何小兽,http://www.tianya.cn/34099118,98506,507,2010-04-07 17:50
1,"你想让汉语纯的像处女一样吗?——也谈无知的""NBA"",""CBA""封杀",/post-free-1857987-1.shtml,xuxiao0519,http://www.tianya.cn/35320813,55311,416,2010-04-07 17:00
2,"CCTV突然封杀NBA,惹千万球迷愤怒",/post-free-1270973-1.shtml,zbgtao,http://www.tianya.cn/14811793,30344,433,2008-05-25 11:51
3,"NBA屏蔽了,""CCTV""呢?",/post-free-1857449-1.shtml,何仁勇,http://www.tianya.cn/7867608,27150,334,2010-04-07 09:59
4,"cctv封杀NBA,大家都很捉急嘛,小小透一下",/post-free-1271933-1.shtml,沟是挤出来的,http://www.tianya.cn/7029557,26300,265,2008-05-25 21:33


In [8]:
len(df)

400

## ——生成帖子列表

In [9]:
df=df.rename(columns = {0:'title', 1:'link', 2:'author',3:'author_page', 4:'click', 5:'reply', 6:'time'})
df[:10]

,title,link,author,author_page,click,reply,time
0,"从抵制家乐福到排斥NBA,中国又不高兴了?",/post-free-1858023-1.shtml,何小兽,http://www.tianya.cn/34099118,98506,507,2010-04-07 17:50
1,"你想让汉语纯的像处女一样吗?——也谈无知的""NBA"",""CBA""封杀",/post-free-1857987-1.shtml,xuxiao0519,http://www.tianya.cn/35320813,55311,416,2010-04-07 17:00
2,"CCTV突然封杀NBA,惹千万球迷愤怒",/post-free-1270973-1.shtml,zbgtao,http://www.tianya.cn/14811793,30344,433,2008-05-25 11:51
3,"NBA屏蔽了,""CCTV""呢?",/post-free-1857449-1.shtml,何仁勇,http://www.tianya.cn/7867608,27150,334,2010-04-07 09:59
4,"cctv封杀NBA,大家都很捉急嘛,小小透一下",/post-free-1271933-1.shtml,沟是挤出来的,http://www.tianya.cn/7029557,26300,265,2008-05-25 21:33
5,"有关部门屏蔽""NBA"",""CBA""和""F1""",/post-free-1858056-1.shtml,ommp,http://www.tianya.cn/8185191,22104,221,2010-04-07 18:31
6,"不是每个人都懂NBA,那有又多少人懂CCTV呢?",/post-free-1857993-1.shtml,李诺言一世,http://www.tianya.cn/26387329,17095,124,2010-04-07 17:04
7,"一个南方人笑看央视总将""NBA""说成""嗯必~诶""",/post-free-1051414-1.shtml,龙起头,http://www.tianya.cn/9500462,15005,405,2007-11-18 18:15
8,推测:中华人民共和国中央电视台为何封杀NBA,/post-free-1270514-1.shtml,hebe__pirlo,http://www.tianya.cn/12766691,14091,118,2008-05-25 00:47
9,"滚开,什么世界杯!不要影响我看NBA总决赛!",/post-free-696485-1.shtml,JJ不再雄赳赳,http://www.tianya.cn/6966288,12182,366,2006-05-17 11:25


## 2.2

In [10]:
def author_crawler(url, file_name):
    try:
        content = urllib2.urlopen(url).read() #获取网页的html文本
        soup = BeautifulSoup(content, "lxml")
        link_info = soup.find_all('div', {'class', 'link-box'})
        followed_num, fans_num = [i.a.text for i in link_info]
        try:
            activity = soup.find_all('span', {'class', 'subtitle'})
            post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
        except:
            post_num, reply_num = 1, 0
        record =  '\t'.join([url, followed_num, fans_num, post_num, reply_num])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record.encode('utf-8')+"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception, e:
        print e, url
        record =  '\t'.join([url, 'na', 'na', 'na', 'na'])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record.encode('utf-8')+"\n") ##!!encode here to utf-8 to avoid encoding
        pass

In [11]:
for k, url in enumerate(df.author_page):
    if k % 50==0:
        print k
    author_crawler(url, '/Users/zhangyixin/Documents/tianya_bbs_threads_author_info.txt')

0
need more than 0 values to unpack http://www.tianya.cn/15026460
need more than 0 values to unpack http://www.tianya.cn/16276251
50
need more than 0 values to unpack http://www.tianya.cn/5280499
100
need more than 0 values to unpack http://www.tianya.cn/16736899
need more than 0 values to unpack http://www.tianya.cn/59385801
150
need more than 0 values to unpack http://www.tianya.cn/98529354
need more than 0 values to unpack http://www.tianya.cn/14253356
sequence item 3: expected string or Unicode, int found http://www.tianya.cn/13243496
sequence item 3: expected string or Unicode, int found http://www.tianya.cn/17462639
sequence item 3: expected string or Unicode, int found http://www.tianya.cn/54504013
200
need more than 0 values to unpack http://www.tianya.cn/61827737
sequence item 3: expected string or Unicode, int found http://www.tianya.cn/13611171
sequence item 3: expected string or Unicode, int found http://www.tianya.cn/13611171
sequence item 3: expected string or Unicode, in

In [12]:
df.author_page[:5]

0    http://www.tianya.cn/34099118
1    http://www.tianya.cn/35320813
2    http://www.tianya.cn/14811793
3     http://www.tianya.cn/7867608
4     http://www.tianya.cn/7029557
Name: author_page, dtype: object

In [13]:
url = df.author_page[1]
content = urllib2.urlopen(url).read() 
soup1 = BeautifulSoup(content, "lxml") 

## ——获得主贴作者信息

In [14]:
user_info = soup1.find('div',  {'class', 'userinfo'})('p')
area, nid, freq_use, last_login_time, reg_time = [i.get_text()[4:] for i in user_info]
print area, nid, freq_use, last_login_time, reg_time

link_info = soup1.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
print followed_num, fans_num

activity = soup1.find_all('span', {'class', 'subtitle'})
post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
print post_num, reply_num

福建厦门市 2100 296 2015-12-02 09:18:38 2010-04-03 14:12:00
17 15
223 322


## 2.3

In [15]:
df.link[0]
url = 'http://bbs.tianya.cn' + df.link[0]
url

'http://bbs.tianya.cn/post-free-1858023-1.shtml'

In [16]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://bbs.tianya.cn/post-free-1270973-1.shtml width=1000 height=500></iframe>')

In [17]:
post = urllib2.urlopen(url).read() 
post_soup = BeautifulSoup(post, "lxml")
print (post_soup.prettify())[:1000]

<!DOCTYPE HTML>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   从抵制家乐福到排斥NBA，中国又不高兴了？_天涯杂谈_天涯论坛
  </title>
  <meta content="从抵制家乐福到排斥NBA，中国又不高兴了？　　08年奥运会期间，因为火炬传递在巴黎受阻，不少爱国人士走向街道，挥着国旗，喊着口号，声泪俱下地哀叹中华民族的曲折命运。爱国人士俨然希望，这样的盛会是中国崛起的一次演出机会，需要通过这样的仪式，来回应文明古国的觉醒，所以，在此期间，任何的..." name="description"/>
  <meta content="IE=EmulateIE9" http-equiv="X-UA-Compatible"/>
  <meta content="何小兽" name="author"/>
  <meta content="format=xhtml; url=http://bbs.tianya.cn/m/post-free-1858023-1.shtml" http-equiv="mobile-agent"/>
  <link href="http://static.tianyaui.com/global/ty/TY.css" rel="stylesheet" type="text/css"/>
  <link href="http://static.tianyaui.com/global/bbs/web/static/css/bbs_article_c55fffc.css" rel="stylesheet" type="text/css"/>
  <link href="http://static.tianyaui.com/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <link href="http://bbs.tianya.cn/post-free-1858023-2.shtml" rel="next"/>
  <script type="text/javascript">
   var bbsGlobal = {
	isEhomeItem : false,
	isNew


In [18]:
pa = post_soup.find_all('div', {'class', 'atl-item'})
len(pa)

95

In [19]:
print pa[0].find('div', {'class', 'bbs-content'}).text.strip()

08年奥运会期间，因为火炬传递在巴黎受阻，不少爱国人士走向街道，挥着国旗，喊着口号，声泪俱下地哀叹中华民族的曲折命运。爱国人士俨然希望，这样的盛会是中国崛起的一次演出机会，需要通过这样的仪式，来回应文明古国的觉醒，所以，在此期间，任何的干扰、质疑，甚至是微量的不怀好意都会将其视为敌意。事实正是如此，那时官方与民间建立起的默契令人惊叹，不仅反对声音得以批量处理，国民对国家、对社会的不满情绪，经过奥运会的组织过滤，似乎也变得可有可无。　　　　火炬在巴黎的传递过程中，这样的默契显然达到了高潮。国旗成了稀缺商品，华人走上街头，声援奥运，他们一边表达着奥运会与政治无关的诉求，一边又不可救药地卷入到这场政治旋涡中。更为突出的是，国内部分爱国人士，不同的省份，不同的人群，都加入到抵制家乐福这场运动中。一家跨国企业因为北京奥运会而卷入到政治斗争中，一年过去了，当我们再回忆起当时的情景，或许不同信仰的人仍然会表达自己的执着与分歧。从功利的角度讲，抵制家乐福运动确实为北京奥运会的成功举办铺平了道路，民间投入的数倍于官方的压力对西方产生的震慑作用非凡。而且，直到如今，我也愿意相信，多数爱国人士之所以卤莽地选择抵制家乐福，是因为鼠目寸光地想追求北京奥运会的成功举办，是对数千年来中国人企图改变苦大仇深、受尽欺凌局面的本能诉求。　　　　但是，北京奥运会显然不是抵制家乐福的全部，人们在反思这样一场运动时，联系到的词汇仍然不够光彩，比如非理性的爱国主义，类似于中国不高兴式的冷战思维，等等。　　　　如果说，奥运会期间抵制家乐福是一次来自民间的政治诉求的宣泄，一次政治热情的普及与发扬，是非理性的、自发的，那么，近来广电总局对禁用NBA等略缩词的规定则显然是官方长期斟酌后的“理性”选择。联想到奥运会期间民间抵制家乐福运动，那时官方的态度是相对暧昧的，既不反对也不支持，提倡民众理性爱国，现在则不同，官方似乎完全出自一相情愿，用一己之力号召人们开始新的生活方式。这样的决策事先无任何征兆，既没有来自NBA方面的挑衅，也也没有字母缩写引发的问题，所以它一出来网上便一片哗然，不动声色地，我们的社会就进入了“无缩写”时代。　　　　让人不解的是，广电总局对此规定竟然没有任何解释，而只是列举了部分荒诞的操作程序，以及主持人“感觉良好”的后续反应。我愿意相信，“无缩写”时代将有助于常识的普及，方便各个阶层的人了解“NBA

In [20]:
pa[1].a

<a class="js-vip-check" href="http://www.tianya.cn/33178761" target="_blank" uid="33178761" uname="zldc1710">zldc1710</a>

In [21]:
print pa[0].find('a', class_ = 'reportme a-link')
print pa[0].find('a', class_ = 'reportme a-link')['replytime']
print pa[0].find('a', class_ = 'reportme a-link')['author']

<a author="何小兽" authorid="34099118" class="reportme a-link" href="javascript:void(0);" replyid="0" replytime="2010-04-07 17:50:00"> 举报</a>
2010-04-07 17:50:00
何小兽


## ——生成回帖信息

In [22]:
for i in pa[:20]:
    p_info = i.find('a', class_ = 'reportme a-link')
    p_time = p_info['replytime']
    p_author_id = p_info['authorid']
    p_author_name = p_info['author']
    p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
    p_content = p_content.replace('\t', '')
    print p_time, '--->', p_author_id, '--->', p_author_name,'--->', p_content, '\n'

2010-04-07 17:50:00 ---> 34099118 ---> 何小兽 ---> 08年奥运会期间，因为火炬传递在巴黎受阻，不少爱国人士走向街道，挥着国旗，喊着口号，声泪俱下地哀叹中华民族的曲折命运。爱国人士俨然希望，这样的盛会是中国崛起的一次演出机会，需要通过这样的仪式，来回应文明古国的觉醒，所以，在此期间，任何的干扰、质疑，甚至是微量的不怀好意都会将其视为敌意。事实正是如此，那时官方与民间建立起的默契令人惊叹，不仅反对声音得以批量处理，国民对国家、对社会的不满情绪，经过奥运会的组织过滤，似乎也变得可有可无。　　　　火炬在巴黎的传递过程中，这样的默契显然达到了高潮。国旗成了稀缺商品，华人走上街头，声援奥运，他们一边表达着奥运会与政治无关的诉求，一边又不可救药地卷入到这场政治旋涡中。更为突出的是，国内部分爱国人士，不同的省份，不同的人群，都加入到抵制家乐福这场运动中。一家跨国企业因为北京奥运会而卷入到政治斗争中，一年过去了，当我们再回忆起当时的情景，或许不同信仰的人仍然会表达自己的执着与分歧。从功利的角度讲，抵制家乐福运动确实为北京奥运会的成功举办铺平了道路，民间投入的数倍于官方的压力对西方产生的震慑作用非凡。而且，直到如今，我也愿意相信，多数爱国人士之所以卤莽地选择抵制家乐福，是因为鼠目寸光地想追求北京奥运会的成功举办，是对数千年来中国人企图改变苦大仇深、受尽欺凌局面的本能诉求。　　　　但是，北京奥运会显然不是抵制家乐福的全部，人们在反思这样一场运动时，联系到的词汇仍然不够光彩，比如非理性的爱国主义，类似于中国不高兴式的冷战思维，等等。　　　　如果说，奥运会期间抵制家乐福是一次来自民间的政治诉求的宣泄，一次政治热情的普及与发扬，是非理性的、自发的，那么，近来广电总局对禁用NBA等略缩词的规定则显然是官方长期斟酌后的“理性”选择。联想到奥运会期间民间抵制家乐福运动，那时官方的态度是相对暧昧的，既不反对也不支持，提倡民众理性爱国，现在则不同，官方似乎完全出自一相情愿，用一己之力号召人们开始新的生活方式。这样的决策事先无任何征兆，既没有来自NBA方面的挑衅，也也没有字母缩写引发的问题，所以它一出来网上便一片哗然，不动声色地，我们的社会就进入了“无缩写”时代。　　　　让人不解的是，广电总局对此规定竟然没有任何解释，而只是列举了部分荒诞的操作程序，以及主持人“

In [23]:
post_soup.find('div', {'class', 'atl-pages'})
post_pages = post_soup.find('div', {'class', 'atl-pages'})
post_pages = post_pages.form['onsubmit'].split(',')[-1].split(')')[0]
post_pages

'6'

In [24]:
url = 'http://bbs.tianya.cn' + df.link[0]
url_base = ''.join(url.split('-')[:-1]) + '-%d.shtml'
url_base

'http://bbs.tianya.cn/postfree1858023-%d.shtml'

In [25]:
def parsePage(pa):
    records = []
    for i in pa:
        p_info = i.find('a', class_ = 'reportme a-link')
        p_time = p_info['replytime']
        p_author_id = p_info['authorid']
        p_author_name = p_info['author']
        p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
        p_content = p_content.replace('\t', '').replace('\n', '')#.replace(' ', '')
        record = p_time + '\t' + p_author_id+ '\t' + p_author_name + '\t'+ p_content
        records.append(record)
    return records

import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [26]:

import random
import time

def crawler(url, file_name):
    try:
        url_1 = 'http://bbs.tianya.cn' + url
        content = urllib2.urlopen(url_1).read() 
        post_soup = BeautifulSoup(content, "lxml") 
        post_form = post_soup.find('div', {'class', 'atl-pages'})
        if post_form.form:
            post_pages = post_form.form['onsubmit'].split(',')[-1].split(')')[0]
            post_pages = int(post_pages)
            url_base = '-'.join(url_1.split('-')[:-1]) + '-%d.shtml'
        else:
            post_pages = 1
        pa = post_soup.find_all('div', {'class', 'atl-item'})
        records = parsePage(pa)
        with open(file_name,'a') as p: 
            for record in records:    
                p.write('1'+ '\t' + url + '\t' + record.encode('utf-8')+"\n") 
        # for the 2nd+ pages
        if post_pages > 1:
            for page_num in range(2, post_pages+1):
                time.sleep(random.random())
                flushPrint(page_num)
                url2 =url_base  % page_num
                content = urllib2.urlopen(url2).read()
                post_soup = BeautifulSoup(content, "lxml") 
                pa = post_soup.find_all('div', {'class', 'atl-item'})
                records = parsePage(pa)
                with open(file_name,'a') as p: 
                    for record in records:    
                        p.write(str(page_num) + '\t' +url + '\t' + record.encode('utf-8')+"\n") 
        else:
            pass
    except Exception, e:
        print e
        pass

In [28]:
for k, link in enumerate(df.link):
    flushPrint(link)
    if k % 10== 0:
        print 'This it the post of : ' + str(k)
    file_name = '/Users/zhangyixin/Documents/tianya_bbs_threads_network.txt'
    crawler(link, file_name)

/post-free-1858023-1.shtmlThis it the post of : 0
/post-free-901420-1.shtmlThis it the post of : 10
/post-free-1269031-1.shtmlThis it the post of : 20
/post-free-817493-1.shtmlThis it the post of : 30
/post-free-1269183-1.shtmlThis it the post of : 40
/post-free-1271571-1.shtmlThis it the post of : 50
/post-free-645114-1.shtmlThis it the post of : 60
/post-free-2051240-1.shtmlThis it the post of : 70
/post-free-1276727-1.shtmlThis it the post of : 80
/post-free-1296640-1.shtmlThis it the post of : 90
/post-free-1475191-1.shtmlThis it the post of : 100
/post-free-1549958-1.shtmlThis it the post of : 110
/post-free-2002827-1.shtmlThis it the post of : 120
/post-free-1158933-1.shtmlThis it the post of : 130
/post-free-2408099-1.shtmlThis it the post of : 140
/post-free-1292703-1.shtmlThis it the post of : 150
/post-free-1693280-1.shtmlThis it the post of : 160
/post-free-1054520-1.shtmlThis it the post of : 170
/post-free-5282263-1.shtmlThis it the post of : 180
/post-free-1276800-1.shtml

In [30]:
dtt = []
with open('/Users/zhangyixin/Documents/tianya_bbs_threads_network.txt', 'r') as f:
    for line in f:
        pnum, link, time, author_id, author, content = line.replace('\n', '').split('\t')
        dtt.append([pnum, link, time, author_id, author, content])
len(dtt)

8373

In [31]:
dt = pd.DataFrame(dtt)
dt[:5]

,0,1,2,3,4,5
0,1,/post-free-1858023-1.shtml,2010-04-07 17:50:00,34099118,何小兽,08年奥运会期间，因为火炬传递在巴黎受阻，不少爱国人士走向街道，挥着国旗，喊着口号，声泪俱下...
1,1,/post-free-1858023-1.shtml,2010-04-07 17:55:38,33178761,zldc1710,哎 人啊人 功利心太重
2,1,/post-free-1858023-1.shtml,2010-04-07 19:16:06,22689153,胡说一句封杀十年,！！！
3,1,/post-free-1858023-1.shtml,2010-04-07 19:17:32,35391908,lfzxxx,"中国啊中国, 你这个算什么,看看这个吧 http://pindao.huoban.tao..."
4,1,/post-free-1858023-1.shtml,2010-04-07 19:19:20,13640263,有事找联合国,好文


In [32]:
dt=dt.rename(columns = {0:'page_num', 1:'link', 2:'time', 3:'author',4:'author_name', 5:'reply'})
dt[:5]

,page_num,link,time,author,author_name,reply
0,1,/post-free-1858023-1.shtml,2010-04-07 17:50:00,34099118,何小兽,08年奥运会期间，因为火炬传递在巴黎受阻，不少爱国人士走向街道，挥着国旗，喊着口号，声泪俱下...
1,1,/post-free-1858023-1.shtml,2010-04-07 17:55:38,33178761,zldc1710,哎 人啊人 功利心太重
2,1,/post-free-1858023-1.shtml,2010-04-07 19:16:06,22689153,胡说一句封杀十年,！！！
3,1,/post-free-1858023-1.shtml,2010-04-07 19:17:32,35391908,lfzxxx,"中国啊中国, 你这个算什么,看看这个吧 http://pindao.huoban.tao..."
4,1,/post-free-1858023-1.shtml,2010-04-07 19:19:20,13640263,有事找联合国,好文


In [33]:
dt.reply[:100]

0     08年奥运会期间，因为火炬传递在巴黎受阻，不少爱国人士走向街道，挥着国旗，喊着口号，声泪俱下...
1                                        哎   人啊人  功利心太重
2                                                   ！！！
3     中国啊中国,　　你这个算什么,看看这个吧　　http://pindao.huoban.tao...
4                                                    好文
5                                                  DING
6                                                说的太好了！
7                                          中国不高兴，后果很严重。
8                                                  钩钩小手
9     而且，直到如今，我也愿意相信，多数爱国人士之所以卤莽地选择抵制家乐福，是因为鼠目寸光地想追求...
10                                                    顶
11                                                    顶
12                                 原来如此啊，广电总局怎么老是跑出来丢人勒
13                      抵制家乐福和排斥NBA挨的上边么?　　　　不要上纲上线什么的!
14                                             Sb廣電總局~~
15    广告/封杀1年/中原枪手-- izydxlw21-- 操作时间:2010-4-7 21:59...
16                                           楼主扯淡，鉴定完毕。
17    LZ 肯定是洋文8级　　　　尔等P民 4级都没~~~ 　　　　你要是怕ABCD没处用